In [1]:
# Perceptron Algorithm on the SONAR Dataset

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from random import seed
from random import randrange
import matplotlib.pyplot as plt


In [22]:
# Loading the SONAR dataset

df = pd.read_csv(r'C:\Users\hunte\OneDrive\Documents\Projects\Perceptron Algorithm\sonar.all-data.csv', header=None)
print("\nSONAR Dataset Shape: {shape}".format(shape=df.shape))

df.rename(columns={df.columns[60]:"Label"}, inplace=True)
df.Label = df.Label.astype('category')
df.head()


SONAR Dataset Shape: (208, 61)


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,Object
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [23]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

        
# Convert string column to integer
def str_column_to_int(dataset, classifier_column):
    class_values = [row[classifier_column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    
    for i, value in enumerate(unique):
        lookup[value] = i
    
    for row in dataset:
        row[classifier_column] = lookup[row[classifier_column]]
    
    print(lookup)
    
    # return Lookup

In [24]:
# Split a dataset into k folds

def cross_validation_split(dataset, number_of_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / number_of_folds)
    
    for i in range(number_of_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    
    return dataset_split

In [25]:
# Calculate accuracy percentage

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


In [26]:
# Evaluate an algorithm using a cross validation split

def evaluate_algorithm(dataset, algorithm, number_of_folds, *args):
    
    folds = cross_validation_split(dataset, number_of_folds)
    scores = list()
    
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    
    return scores

In [27]:
# Make a prediction with weights

def predict(row, weights):

    activation = weights[0]
    for i in range(len(row) - 1):
        activation += weights[i+1] * row[i]
    
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights

def train_weights(train, learning_rate, number_of_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    
    for epoch in range(number_of_epoch):
        for row in train:
            prediction = prediction(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + learning_rate * error
            
            for i in range(len(row) - 1):
                weights[i+1] = weights[i+1] + learning_rate * error * row[i]
    
    return weights

# Perceptron Algorithm with Stochastic Gradient Descent

def perceptron(train, test, learning_rate, number_of_epoch):
    predictions = list()
    weights = train_weights(train, learning_rate, number_of_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return (predictions)

In [28]:
# Test the Perceptron algorithm on the SONAR dataset
seed(1)

# Load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)

# Convert all values into one data type
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

# Convert string class to integers
str_column_to_int(dataset, len(dataset[0]) - 1)

# Evaluate algorithm
number_of_folds = 2
learning_rate = 0.01
number_of_epoch = 400
scores = evaluate_algorithm(dataset, perceptron, number_of_folds, learning_rate, number_of_epoch)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
print("")

scores2 = evaluate_algorithm(dataset, perceptron, number_of_folds, learning_rate, number_of_epoch)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores2))))
print("")

scores3 = evaluate_algorithm(dataset, perceptron, number_of_folds, learning_rate, number_of_epoch)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores3))))
print("")


NameError: name 'load_csv' is not defined